<a href="https://colab.research.google.com/github/KyleEtera/MAT422/blob/main/3_2_Continuity_and_Differentiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [117]:
import numpy as np, pandas as pd, scipy.linalg as la, matplotlib.pyplot as plt
import sympy as sp
from sympy.abc import rho, phi
from sympy.matrices import *

**3.2.1. Limits and Continuity** - We say we have found the Limit of a function as it's variable approaches any given value if we can find a difference between the limit evaluated at the x value subtracted or added by a small value, ϵ, and the function being smaller than ϵ itself. The value being approached can be any value, often 0 is used or the most interesting option.

We can add onto this that the function may be called continuous if the limit equals the value at the x value. With a continuous function, we can combine multiple functions into one without a problem.

For example, multiply a value that has been added to, so 2*(3+20) is a valid operation represented by g(f(x)) where g(x) = 2x, f(x) = 3+x and x = 20.

We can find an maximum or minimum value if the value is greater than or less than all other values for f(x).

In [ ]:
x = sp.Symbol('x')
f = 25 / (x**2+x+5)
L = sp.limit(f, x, 0)
print(L)

f = (x+2) / (-x**4+x**3-x**2+x+14)
L = sp.limit(f, x, 4)
print(L)

5
-3/95


**3.2.2. Derivatives** - For single variables, we can find the derivative by taking f'(x) = df(x)/dx = lim f(x+h)-f(x) / h as h approaches 0.

For multiple variables, we find the derivative in a similar way, but with respect to the particular variable we are interested in. If we have two variables and we take the derivative with respect to either variable and then the other, we get the same values. So, if we have a f(x,y), fₓ(x,y) and fᵧ(x,y) may be different, but fₓᵧ(x,y) and fᵧₓ(x,y) will be the same.

The Jacobian of f produces a matrix of column vectors for each x value with their respective partial derivative.

In [118]:
x=sp.Symbol('x')
y=sp.Symbol('y')
f = x**2+x+5
df=sp.diff(f,x)
ddf=sp.diff(df,x)
print(f, ' |\t', df, '  |', ddf)

f = (sp.sin(y)) / (sp.cos(x))

dfdx = sp.Derivative(f,x)
dfdy = sp.Derivative(f,y)
dfdxy = sp.Derivative(dfdx,y)
dfdyx = sp.Derivative(dfdy,x)
print(f, '|', dfdx.doit(), '|', dfdy.doit(), '|', dfdxy.doit(), '|', dfdyx.doit())

X = sp.Matrix([rho*sp.cos(phi), rho*sp.sin(phi), rho**2])
Y = sp.Matrix([rho, phi])

print(X[0], '\n', X[1], '\n', X[2])
X.jacobian(Y)

x**2 + x + 5  |	 2*x + 1   | 2
sin(y)/cos(x) | sin(x)*sin(y)/cos(x)**2 | cos(y)/cos(x) | sin(x)*cos(y)/cos(x)**2 | sin(x)*cos(y)/cos(x)**2
rho*cos(phi) 
 rho*sin(phi) 
 rho**2


⎡cos(φ)  -ρ⋅sin(φ)⎤
⎢                 ⎥
⎢sin(φ)  ρ⋅cos(φ) ⎥
⎢                 ⎥
⎣ 2⋅ρ        0    ⎦

**3.2.3. Taylor’s Theorem** - In the case we have a differentiable function around a given point [a,b], we can approximate it with a polynomial of the form f(b) = f(a) + (b-a)f'(a) + 1/2(b-a)^2f''(a) + ... + (b-a)^(m-1)/(m-1)!f^(m-1)(a) + Rₘ where m is the number of times the derivatives is continuous for.

In [119]:
x, t, a, h, k = sp.symbols('x t a h k')
x0, t0 = sp.symbols('x_0, t_0')

v = sp.Function('v')(x, t)
X = Matrix([x,t])
dX = Matrix([h,k])
sp.init_printing()
dX

⎡h⎤
⎢ ⎥
⎣k⎦

In [102]:
X

⎡x⎤
⎢ ⎥
⎣t⎦

In [103]:
TE = 0
v0 = v
for idx, x in enumerate(X):
    v0 = v0.subs(x, 0)
TE += v0
TE

v(0, 0)

In [104]:
gradV = Matrix([v.diff(xi,1) for xi in X])
gradV

⎡∂          ⎤
⎢──(v(x, t))⎥
⎢∂x         ⎥
⎢           ⎥
⎢∂          ⎥
⎢──(v(x, t))⎥
⎣∂t         ⎦

In [105]:
firstTerm = gradV.T*dX
TE += sum(firstTerm)
firstTerm[0]

  ∂               ∂          
h⋅──(v(x, t)) + k⋅──(v(x, t))
  ∂x              ∂t         

In [106]:
def factorial(n):
    if n <= 0:
        return 1
    else:
        return n * factorial(n - 1)
tsOrderGroup = lambda f, X, n: dX.T * Matrix([f.diff(xi,1) for xi in X]) / factorial(n)
grad = tsOrderGroup(v, X, 1).T
fir = grad
sum(fir)

  ∂               ∂          
h⋅──(v(x, t)) + k⋅──(v(x, t))
  ∂x              ∂t         

In [107]:
sp.hessian(v, X)
secondTerm = .5*dX.T*sp.hessian(v, X)*dX
TE += sum(secondTerm)
sp.simplify(sum(secondTerm))

        2                       2                      2          
     2 ∂                       ∂                    2 ∂           
0.5⋅h ⋅───(v(x, t)) + 1.0⋅h⋅k⋅─────(v(x, t)) + 0.5⋅k ⋅───(v(x, t))
         2                    ∂x ∂t                     2         
       ∂x                                             ∂t          

In [108]:
gNext = Matrix([tsOrderGroup(dv, X, 2) for dv in grad * factorial(2 - 1)])
sp.simplify(sum(gNext))

    2                                      2          
 2 ∂                                    2 ∂           
h ⋅───(v(x, t))                        k ⋅───(v(x, t))
     2                  2                   2         
   ∂x                  ∂                  ∂t          
─────────────── + h⋅k⋅─────(v(x, t)) + ───────────────
       2              ∂x ∂t                   2       

In [110]:
gNext = Matrix([tsOrderGroup(dv, X, 3) for dv in gNext * factorial(3 - 1)])
TE += sum(gNext)
sp.simplify(sum(gNext))

    4                       4                       4                         4                  4 ↪
 4 ∂                 3     ∂                2  2   ∂                     3   ∂                4 ∂  ↪
h ⋅───(v(x, t))   2⋅h ⋅k⋅──────(v(x, t))   h ⋅k ⋅───────(v(x, t))   2⋅h⋅k ⋅──────(v(x, t))   k ⋅── ↪
     4                     3                       2   2                        3                  ↪
   ∂x                    ∂x  ∂t                  ∂x  ∂t                    ∂x ∂t                ∂t ↪
─────────────── + ────────────────────── + ────────────────────── + ────────────────────── + ───── ↪
      18                    9                        3                        9                    ↪

↪           
↪           
↪ ─(v(x, t))
↪ 4         
↪           
↪ ──────────
↪  18       

In [111]:
TE += sp.O(h**4) + sp.O(k**4)
sp.simplify(TE)

                                          2                                   3                    ↪
                    ∂                  2 ∂                                 3 ∂                     ↪
1.0⋅v(0, 0) + 1.0⋅k⋅──(v(x, t)) + 0.5⋅k ⋅───(v(x, t)) + 0.166666666666667⋅k ⋅───(v(x, t)) + 1.0⋅h⋅ ↪
                    ∂t                     2                                   3                   ↪
                                         ∂t                                  ∂t                    ↪

↪                         2                          3                                        4    ↪
↪ ∂                      ∂                      2   ∂                                    3   ∂     ↪
↪ ──(v(x, t)) + 1.0⋅h⋅k⋅─────(v(x, t)) + 0.5⋅h⋅k ⋅──────(v(x, t)) + 0.222222222222222⋅h⋅k ⋅──────( ↪
↪ ∂x                    ∂x ∂t                          2                                        3  ↪
↪                                                 ∂x ∂t                                   

In [112]:
TE.removeO()

  ⎛  ⎛  ⎛   4                  4           ⎞     ⎛     4                   4            ⎞⎞     ⎛   ↪
  ⎜  ⎜  ⎜  ∂                  ∂            ⎟     ⎜    ∂                   ∂             ⎟⎟     ⎜   ↪
  ⎜h⋅⎜h⋅⎜h⋅───(v(x, t)) + k⋅──────(v(x, t))⎟ + k⋅⎜h⋅──────(v(x, t)) + k⋅───────(v(x, t))⎟⎟   k⋅⎜h⋅ ↪
  ⎜  ⎜  ⎜    4                3            ⎟     ⎜    3                   2   2         ⎟⎟     ⎜   ↪
  ⎜  ⎝  ⎝  ∂x               ∂x  ∂t         ⎠     ⎝  ∂x  ∂t              ∂x  ∂t          ⎠⎠     ⎝   ↪
h⋅⎜─────────────────────────────────────────────────────────────────────────────────────── + ───── ↪
  ⎝                                           3                                                    ↪
────────────────────────────────────────────────────────────────────────────────────────────────── ↪
                                                                                            6      ↪
                                                                                           

In [113]:
def myTaylorSeries(v, nOrder, X, dX):
    TE = 0
    v0 = v
    for idx, x in enumerate(X):
        v0 = v0.subs(x, 0)
        TE += sp.O(x**(nOrder+1))
    TE += v0
    tsOrderGroup = lambda f, X, n: dX.T * Matrix([f.diff(xi,1) for xi in X])
    grad = tsOrderGroup(v, X, 1).T
    TE += sum(grad)
    gNext = grad
    for nord in range(2, nOrder+1):
        gNext = Matrix([tsOrderGroup(dv, X, nord) for dv in gNext])
        TE += sum(gNext)/factorial(nord)
    return TE
sp.simplify(myTaylorSeries(v, 4, X, dX))

                              2                 3                 4                                ↪
                           2 ∂               3 ∂               4 ∂                                 ↪
                          k ⋅───(v(x, t))   k ⋅───(v(x, t))   k ⋅───(v(x, t))                      ↪
                               2                 3                 4                               ↪
            ∂                ∂t                ∂t                ∂t               ∂                ↪
v(0, 0) + k⋅──(v(x, t)) + ─────────────── + ─────────────── + ─────────────── + h⋅──(v(x, t)) + h⋅ ↪
            ∂t                   2                 6                24            ∂x               ↪

↪                            3                      4                  2                     3     ↪
↪                       2   ∂                  3   ∂                2 ∂               2     ∂      ↪
↪                    h⋅k ⋅──────(v(x, t))   h⋅k ⋅──────(v(x, t))   h ⋅───(v(x, t))   h ⋅k⋅

In [114]:
x, y, z, t, h, l, m, k = sp.symbols('x, y, z, t, h, l, m, k')
S = Matrix([x, y, z, t])
dS = Matrix([h, l, m, k])
f = sp.Function('f')(x, y, z, t)
fTS = myTaylorSeries(f, 2, S, dS)
sp.simplify(fTS)

                                          2                                                        ↪
                                       2 ∂                                                         ↪
                                      m ⋅───(f(x, y, z, t))                                        ↪
                                           2                                              2        ↪
                  ∂                      ∂z                     ∂                        ∂         ↪
f(0, 0, 0, 0) + m⋅──(f(x, y, z, t)) + ───────────────────── + l⋅──(f(x, y, z, t)) + l⋅m⋅─────(f(x, ↪
                  ∂z                            2               ∂y                      ∂z ∂y      ↪

↪                  2                                                                               ↪
↪               2 ∂                                                                                ↪
↪              l ⋅───(f(x, y, z, t))                                                      

Source:
https://github.com/madonnaojorin/MAT494/blob/main/3.2_Continunity_and_Differentiation.ipynb